In [1]:
%%sh
pwd

/home/ubuntu/whisper-dev/whisper-stream/development/notebooks


In [2]:
%load_ext autoreload
%load_ext viztracer

In [3]:
import jax

jax.device_count(), jax.devices()[0].device_kind

(1, 'NVIDIA A10G')

In [4]:
from whisper_stream.core.helpers.data_loading import load_data_samples_from_path
from whisper_stream.vendored.whisper_jax import (
    FlaxWhisperPipeline,
)
from whisper_stream.projects.jax_pipelines.constants import (
    JAXValidDtypesMapping,
    JAXScalarDType,
)

from whisper_stream.core.constants import WhisperValidCheckpoints, WhisperValidTasks

from whisper_stream.core.logger import LogLevelNames
from pathlib import Path

%autoreload 2

In [5]:
# Prepare
checkpoint: WhisperValidCheckpoints = "openai/whisper-large-v2"
model_dtype: JAXScalarDType = JAXValidDtypesMapping["BFLOAT16"]
task: WhisperValidTasks = "transcribe"
language: str = "english"
return_timestamps: bool = True
batch_size: int = 1
log_level: LogLevelNames = "INFO"

data_directory = Path("../../data")

run_opts = {
    "batch_size": batch_size,
    "return_timestamps": return_timestamps,
    "language": language,
    "task": task,
}

# construct
pipeline = FlaxWhisperPipeline(
    checkpoint=checkpoint, dtype=model_dtype, batch_size=batch_size
)

In [6]:
# Load data
pipeline_data: bytes = load_data_samples_from_path(
    "audio_2.mp3", directory=data_directory, binary_mode=True
)  # 2s
pipeline_data_large: bytes = load_data_samples_from_path(
    "tryst.mp3", directory=data_directory, binary_mode=True
)  # 4:44s

In [9]:
# initialize & warmup
%time pipeline(pipeline_data, **run_opts)

CPU times: user 487 ms, sys: 553 ms, total: 1.04 s
Wall time: 445 ms


{'text': ' I know all the players of cricket.',
 'chunks': [{'timestamp': (0.0, 2.8),
   'text': ' I know all the players of cricket.'}]}

In [11]:
# should be warmed up now (time should be similar to # small data)
%time pipeline(pipeline_data, **run_opts)

CPU times: user 520 ms, sys: 518 ms, total: 1.04 s
Wall time: 443 ms


{'text': ' I know all the players of cricket.',
 'chunks': [{'timestamp': (0.0, 2.8),
   'text': ' I know all the players of cricket.'}]}

In [12]:
# small data
%time pipeline(pipeline_data, **run_opts)

CPU times: user 467 ms, sys: 575 ms, total: 1.04 s
Wall time: 445 ms


{'text': ' I know all the players of cricket.',
 'chunks': [{'timestamp': (0.0, 2.8),
   'text': ' I know all the players of cricket.'}]}

In [23]:
# small data in batch
%time list([pipeline(data, **run_opts) for data in [pipeline_data] * 10])

CPU times: user 1.91 s, sys: 5 s, total: 6.91 s
Wall time: 1.66 s


[{'text': ' I know all the players of cricket.',
  'chunks': [{'timestamp': (0.0, 2.8),
    'text': ' I know all the players of cricket.'}]},
 {'text': ' I know all the players of cricket.',
  'chunks': [{'timestamp': (0.0, 2.8),
    'text': ' I know all the players of cricket.'}]},
 {'text': ' I know all the players of cricket.',
  'chunks': [{'timestamp': (0.0, 2.8),
    'text': ' I know all the players of cricket.'}]},
 {'text': ' I know all the players of cricket.',
  'chunks': [{'timestamp': (0.0, 2.8),
    'text': ' I know all the players of cricket.'}]},
 {'text': ' I know all the players of cricket.',
  'chunks': [{'timestamp': (0.0, 2.8),
    'text': ' I know all the players of cricket.'}]},
 {'text': ' I know all the players of cricket.',
  'chunks': [{'timestamp': (0.0, 2.8),
    'text': ' I know all the players of cricket.'}]},
 {'text': ' I know all the players of cricket.',
  'chunks': [{'timestamp': (0.0, 2.8),
    'text': ' I know all the players of cricket.'}]},
 {'tex

In [24]:
# chunkable data
%time pipeline(pipeline_data_large, **run_opts)

CPU times: user 3.83 s, sys: 8.25 s, total: 12.1 s
Wall time: 2.38 s


{'text': " long years ago, we made a trick with this penny. And now the time comes when we shall redeem our pledge, not only for in full measure, but there is a potentially. At the stroke of the midnight hour, when the world sleeps, India will awake to light and freedom. A moment comes which come but rarely in history. When we step out from the world to the new, when an agent, and when the throne of a nation, lungs suppressed, fine utterance. It is fitting that at this solemn moment, we take the pledge of dedication to the service of India and her people and to the still larger cause of humanity. At the grown of history India started on her unending quest. And trackless centuries of feud with her spying and the grandeur of her successes and her failures. Who good and in what you know like she has never lost sight of that quest for God in the ideals which gave her strength. We end today a pariet of the infotainment and India discovers herself again. Their achievement we celebrate today 

In [27]:
# chunkable data in batches
%time list([pipeline(data, **run_opts) for data in [pipeline_data_large] * 32])

CPU times: user 2min 1s, sys: 4min 25s, total: 6min 26s
Wall time: 1min 16s


[{'text': " long years ago, we made a trick with this penny. And now the time comes when we shall redeem our pledge, not only for in full measure, but there is a potentially. At the stroke of the midnight hour, when the world sleeps, India will awake to light and freedom. A moment comes which come but rarely in history. When we step out from the world to the new, when an agent, and when the throne of a nation, lungs suppressed, fine utterance. It is fitting that at this solemn moment, we take the pledge of dedication to the service of India and her people and to the still larger cause of humanity. At the grown of history India started on her unending quest. And trackless centuries of feud with her spying and the grandeur of her successes and her failures. Who good and in what you know like she has never lost sight of that quest for God in the ideals which gave her strength. We end today a pariet of the infotainment and India discovers herself again. Their achievement we celebrate today